In [1]:
import pandas as pd
from seirsplus.models import *
import networkx
import pickle

In [24]:
# build pandas dataframe from excel file
filePath = 'google_county_data.xlsx'
activity_data = pd.read_excel(filePath)
# Save file so it does not have to be recompiled
with open('activity_data.pickle', 'wb') as file:
    pickle.dump(activity_data, file)

In [25]:
with open('activity_data.pickle', 'rb') as file:
    activity_data = pickle.load(file)

In [26]:
activity_data.head()

,country_region_code,country_region,sub_region_1,sub_region_2,date,retail_and_recreation_percent_change_from_baseline,grocery_and_pharmacy_percent_change_from_baseline,parks_percent_change_from_baseline,transit_stations_percent_change_from_baseline,workplaces_percent_change_from_baseline,residential_percent_change_from_baseline
0,US,United States,NaN,NaN,2020-02-15,6.0,2.0,15.0,3.0,2.0,-1.0
1,US,United States,NaN,NaN,2020-02-16,7.0,1.0,16.0,2.0,0.0,-1.0
2,US,United States,NaN,NaN,2020-02-17,6.0,0.0,28.0,-9.0,-24.0,5.0
3,US,United States,NaN,NaN,2020-02-18,0.0,-1.0,6.0,1.0,0.0,1.0
4,US,United States,NaN,NaN,2020-02-19,2.0,0.0,8.0,1.0,1.0,0.0


In [63]:
state_list = ['Washington', 'Georgia', 'New Jersey', 'Utah']
files_per_capita = ["data/washington_per_capita.xlsx","data/georgia_per_capita.xlsx","data/new_jersey_per_capita.xlsx","data/per_capita_utah.xlsx"]
files_industry = ['data/CAEMP25N_WA_2001_2018.csv','data/CAEMP25N_GA_2001_2018.csv','data/CAEMP25N_NJ_2001_2018.csv','data/CAEMP25N_UT_2001_2018.csv']

per_capita_dfs = []
industry_dfs = []
travel_data_dfs = []

for fileName in files_per_capita:
    with open(fileName,'rb') as file:
        df = pd.read_excel(file)
        df = df.filter(items=['location','dollars_2018'])
        per_capita_dfs.append(df)

for fileName in files_industry:
    with open(fileName,'rb') as file:
        df = pd.read_csv(file)
        df = df.filter(items=['GeoName','Description','2018'])
        industry_dfs.append(df)

activity_data = activity_data.filter(['country_region','sub_region_1','sub_region_2','date','workplaces_percent_change_from_baseline'])
for state in state_list:
    df  = activity_data.loc[activity_data['sub_region_1'] == state]
    df = df.filter(items=['sub_region_2','date','workplaces_percent_change_from_baseline'])
    travel_data_dfs.append(df)

with open('per_capita_dfs.pickle', 'wb') as file:
    pickle.dump(per_capita_dfs, file)
with open('industry_dfs.pickle', 'wb') as file:
    pickle.dump(industry_dfs, file)

In [64]:
from functools import reduce
def rows_to_cols(relevant_elements, orig_df, colName, colData, idCol):
    dataframes = []
    characteristics = orig_df[colName]
    for element in relevant_elements:
        df = orig_df.copy()
        df = df[characteristics == element]
        # df = df.drop(columns=['Element Name'])
        df = df.drop(columns=[colName])
        # df = df.rename(columns={'Data Value': element})
        df = df.rename(columns={colData: element})
        dataframes.append(df)

    return reduce(lambda df1,df2: pd.merge(df1,df2,on=[idCol], how='left'), dataframes)

In [65]:
for state in range(len(state_list)):
    relevant_elements = ['Total employment (number of jobs)',
' Wage and salary employment',
' Proprietors employment',
'  Farm proprietors employment',
'  Nonfarm proprietors employment 2/',
' Farm employment',
' Nonfarm employment',
'  Private nonfarm employment',
'   Forestry, fishing, and related activities',
'   Mining, quarrying, and oil and gas extraction',
'   Utilities',
'   Construction',
'   Manufacturing',
'   Wholesale trade',
'   Retail trade',
'   Transportation and warehousing',
'   Information',
'   Finance and insurance',
'   Real estate and rental and leasing',
'   Professional, scientific, and technical services',
'   Management of companies and enterprises',
'   Administrative and support and waste management and remediation services',
'   Educational services',
'   Health care and social assistance',
'   Arts, entertainment, and recreation',
'   Accommodation and food services',
'   Other services (except government and government enterprises)',
'  Government and government enterprises',
'   Federal civilian',
'   Military',
'   State and local',
'    State government',
'    Local government'
]
    industry_dfs[state] = rows_to_cols(relevant_elements, industry_dfs[state], 'Description', '2018', 'GeoName')

In [66]:
for state in range(len(state_list)):
    relevant_elements = ['2/15/2020',
'2/16/2020',
'2/17/2020',
'2/18/2020',
'2/19/2020',
'2/20/2020',
'2/21/2020',
'2/22/2020',
'2/23/2020',
'2/24/2020',
'2/25/2020',
'2/26/2020',
'2/27/2020',
'2/28/2020',
'2/29/2020',
'3/1/2020',
'3/2/2020',
'3/3/2020',
'3/4/2020',
'3/5/2020',
'3/6/2020',
'3/7/2020',
'3/8/2020',
'3/9/2020',
'3/10/2020',
'3/11/2020',
'3/12/2020',
'3/13/2020',
'3/14/2020',
'3/15/2020',
'3/16/2020',
'3/17/2020',
'3/18/2020',
'3/19/2020',
'3/20/2020',
'3/21/2020',
'3/22/2020',
'3/23/2020',
'3/24/2020',
'3/25/2020',
'3/26/2020',
'3/27/2020',
'3/28/2020',
'3/29/2020',
'3/30/2020',
'3/31/2020',
'4/1/2020',
'4/2/2020',
'4/3/2020',
'4/4/2020',
'4/5/2020',
'4/6/2020',
'4/7/2020',
'4/8/2020',
'4/9/2020',
'4/10/2020',
'4/11/2020'
]

    travel_data_dfs[state] = rows_to_cols(relevant_elements, travel_data_dfs[state], 'date', 'workplaces_percent_change_from_baseline', 'sub_region_2')
    

In [67]:
per_capita_dfs[0].head()

,location,dollars_2018
0,Washington,62026
1,Adams,42800
2,Asotin,47104
3,Benton,47465
4,Chelan,54763


In [71]:
for state in range(len(state_list)):
    travel_data_dfs[state] = travel_data_dfs[state].drop(travel_data_dfs[state].index[0])
    travel_data_dfs[state]['sub_region_2'] = travel_data_dfs[state]['sub_region_2'].apply(lambda a: a.split(' ')[0])
travel_data_dfs[0].head()

,sub_region_2,2/15/2020,2/16/2020,2/17/2020,2/18/2020,2/19/2020,2/20/2020,2/21/2020,2/22/2020,2/23/2020,...,4/2/2020,4/3/2020,4/4/2020,4/5/2020,4/6/2020,4/7/2020,4/8/2020,4/9/2020,4/10/2020,4/11/2020
2,Asotin,0.0,-1.0,-29.0,3.0,4.0,2.0,4.0,7.0,-1.0,...,-47.0,-43.0,-31.0,-22.0,-40.0,-40.0,-43.0,-43.0,-42.0,-29.0
3,Benton,-1.0,-1.0,-38.0,2.0,1.0,0.0,1.0,0.0,-2.0,...,-54.0,-50.0,-36.0,-40.0,-53.0,-54.0,-55.0,-55.0,-52.0,-36.0
4,Chelan,-4.0,1.0,-29.0,0.0,2.0,0.0,0.0,-1.0,0.0,...,-49.0,-49.0,-44.0,-42.0,-43.0,-47.0,-46.0,-48.0,-50.0,-45.0
5,Clallam,-2.0,1.0,-30.0,1.0,2.0,2.0,2.0,0.0,3.0,...,-47.0,-45.0,-38.0,-38.0,-40.0,-44.0,-44.0,-46.0,-44.0,-34.0
6,Clark,-4.0,-1.0,-27.0,1.0,2.0,2.0,3.0,1.0,0.0,...,-52.0,-51.0,-39.0,-41.0,-50.0,-51.0,-51.0,-51.0,-52.0,-38.0


In [72]:
for state in range(len(state_list)):
    industry_dfs[state] = industry_dfs[state].drop(industry_dfs[state].index[0])
    industry_dfs[state]['GeoName'] = industry_dfs[state]['GeoName'].apply(lambda a: a.split(', ')[0])
industry_dfs[0].head()

,GeoName,Total employment (number of jobs),Wage and salary employment,Proprietors employment,Farm proprietors employment,Nonfarm proprietors employment 2/,Farm employment,Nonfarm employment,Private nonfarm employment,"Forestry, fishing, and related activities",...,Health care and social assistance,"Arts, entertainment, and recreation",Accommodation and food services,Other services (except government and government enterprises),Government and government enterprises,Federal civilian,Military,State and local,State government,Local government
1,Adams,9585,7525,2060,483,1577,1368,8217,6439,450,...,(D),53,474,363,1778,39,49,1690,74,1616
2,Asotin,9402,6988,2414,166,2248,219,9183,7914,(D),...,1691,201,752,695,1269,51,57,1161,149,1012
3,Benton,113284,93867,19417,1349,18068,4932,108352,94589,(D),...,13753,2382,8303,5259,13763,747,522,12494,1617,10877
4,Chelan,56065,45481,10584,715,9869,5161,50904,43529,(D),...,7065,1124,5501,2355,7375,600,193,6582,1191,5391
5,Clallam,34952,25918,9034,446,8588,612,34340,25875,830,...,3460,752,3105,2469,8465,456,552,7457,1178,6279


In [73]:
for state in range(len(state_list)):
    per_capita_dfs[state] = per_capita_dfs[state].rename(columns={'location':'County', 'dollars_2018':'Per Capita Income 2018'})
    travel_data_dfs[state] = travel_data_dfs[state].rename(columns={'sub_region_2':'County'})
    industry_dfs[state] = industry_dfs[state].rename(columns={'GeoName':'County'})

In [79]:
# Combine data into a single dataframe for each given statea
combined_data_dfs = []
for state in range(len(state_list)):
    dataframes = [per_capita_dfs[state], travel_data_dfs[state], industry_dfs[state]]
    # df = reduce(lambda df1,df2: pd.merge(df1,df2,on=['County'], how='left'), dataframes)
    df = reduce(lambda df1, df2: pd.merge(left=df1, right=df2, left_on=['County'], right_on=['County']), dataframes)
    combined_data_dfs.append(df)
with open('combined_data_dfs.pickle', 'wb') as file:
    pickle.dump(combined_data_dfs, file)

## Regression

In [ ]:
import pandas as pd
import pickle

In [82]:
state_list = ['Washington', 'Georgia', 'New Jersey', 'Utah']
with open('combined_data_dfs.pickle', 'rb') as file:
    combined_data_dfs = pickle.load(file)

In [83]:
combined_data_dfs[0].head() # Data for Washington Counties

,County,Per Capita Income 2018,2/15/2020,2/16/2020,2/17/2020,2/18/2020,2/19/2020,2/20/2020,2/21/2020,2/22/2020,...,Health care and social assistance,"Arts, entertainment, and recreation",Accommodation and food services,Other services (except government and government enterprises),Government and government enterprises,Federal civilian,Military,State and local,State government,Local government
0,Asotin,47104,0.0,-1.0,-29.0,3.0,4.0,2.0,4.0,7.0,...,1691,201,752,695,1269,51,57,1161,149,1012
1,Benton,47465,-1.0,-1.0,-38.0,2.0,1.0,0.0,1.0,0.0,...,13753,2382,8303,5259,13763,747,522,12494,1617,10877
2,Chelan,54763,-4.0,1.0,-29.0,0.0,2.0,0.0,0.0,-1.0,...,7065,1124,5501,2355,7375,600,193,6582,1191,5391
3,Clallam,46120,-2.0,1.0,-30.0,1.0,2.0,2.0,2.0,0.0,...,3460,752,3105,2469,8465,456,552,7457,1178,6279
4,Clark,53423,-4.0,-1.0,-27.0,1.0,2.0,2.0,3.0,1.0,...,28560,5468,15118,13118,28533,3482,1257,23794,4140,19654


## Data from each County
* Jobs per industry
* Percentage change in travel to workplace
* Average Income per county

One dataframe for each state
1 col for each industry and employment in each (for each county)
1 col for each date for workplace percentage changes
1 col for avergae income

In [15]:
industry_data.head()

,GeoFIPS,GeoName,Region,TableName,LineCode,IndustryClassification,Description,Unit,2001,2002,...,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018
0,"""13000""",Georgia,5.0,CAEMP25N,10.0,...,Total employment (number of jobs),Number of jobs,4871814,4858288,...,5247022,5211352,5325616,5384572,5496783,5670735,5832372,5971729,6140776,6274450
1,"""13000""",Georgia,5.0,CAEMP25N,20.0,...,Wage and salary employment,Number of jobs,4122141,4075446,...,4074870,4028041,4066303,4123085,4199911,4319223,4438625,4552174,4640639,4727591
2,"""13000""",Georgia,5.0,CAEMP25N,40.0,...,Proprietors employment,Number of jobs,749673,782842,...,1172152,1183311,1259313,1261487,1296872,1351512,1393747,1419555,1500137,1546859
3,"""13000""",Georgia,5.0,CAEMP25N,50.0,...,Farm proprietors employment,Number of jobs,50229,44316,...,37859,37197,36666,35401,35501,35267,35464,36067,35695,35007
4,"""13000""",Georgia,5.0,CAEMP25N,60.0,...,Nonfarm proprietors employment 2/,Number of jobs,699444,738526,...,1134293,1146114,1222647,1226086,1261371,1316245,1358283,1383488,1464442,1511852


In [9]:
activity_data

,country_region_code,country_region,sub_region_1,sub_region_2,date,retail_and_recreation_percent_change_from_baseline,grocery_and_pharmacy_percent_change_from_baseline,parks_percent_change_from_baseline,transit_stations_percent_change_from_baseline,workplaces_percent_change_from_baseline,residential_percent_change_from_baseline
0,US,United States,Georgia,NaN,2020-02-15,5.0,1.0,42.0,4.0,1.0,-1.0
1,US,United States,Georgia,NaN,2020-02-16,-2.0,-5.0,-19.0,-5.0,-3.0,1.0
2,US,United States,Georgia,NaN,2020-02-17,4.0,1.0,24.0,0.0,-19.0,4.0
3,US,United States,Georgia,NaN,2020-02-18,-4.0,-5.0,-23.0,1.0,-5.0,3.0
4,US,United States,Georgia,NaN,2020-02-19,2.0,0.0,2.0,3.0,-2.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...
8203,US,United States,Georgia,Worth County,2020-04-07,NaN,NaN,NaN,NaN,-43.0,NaN
8204,US,United States,Georgia,Worth County,2020-04-08,NaN,NaN,NaN,NaN,-45.0,NaN
8205,US,United States,Georgia,Worth County,2020-04-09,NaN,NaN,NaN,NaN,-45.0,NaN
8206,US,United States,Georgia,Worth County,2020-04-10,NaN,NaN,NaN,NaN,-47.0,NaN


In [16]:
per_capita_data.head()

,Unnamed: 0,dollars_2016,dollars_2017,dollars_2018,dollar_rank_2018,percent_change_2017,percent_change_2018,percent_rank_2018
0,United States,49870.0,51885.0,54446.0,--,4.0,4.9,--
1,Alabama,39224.0,40467.0,42238.0,--,3.2,4.4,--
2,Autauga,39561.0,40450.0,41618.0,10,2.2,2.9,61
3,Baldwin,42907.0,43989.0,45596.0,4,2.5,3.7,55
4,Barbour,31595.0,33048.0,35199.0,41,4.6,6.5,2
